# Areas of Interest Data Analysis

In [40]:
# lib for dataframes
import pandas as pd

from scipy import stats

from chord import Chord

Config

In [41]:
config_prefix = ['BR', 'BI', 'TR', 'TI']

config_datasheet = r'./results/preprocessed.xlsx'

Import Dataframe

In [42]:
df_matrix = []
raw = pd.read_excel(config_datasheet)
df = pd.DataFrame(raw)
df = df.loc[df["Correctness"]==True]

Prepare Image Paths

In [43]:
algo_name_array = [name for name in df["Algorithm"].unique()]
aoi_cat_path_matrix = []

for algo_name in algo_name_array:
    aoi_array =[
        'data/aoi_categorized/AOI_BR_' + algo_name + '.xlsx',
        'data/aoi_categorized/AOI_BI_' + algo_name + '.xlsx',
        'data/aoi_categorized/AOI_TR_' + algo_name + '.xlsx',
        'data/aoi_categorized/AOI_TI_' + algo_name + '.xlsx',
    ]
    aoi_cat_path_matrix.append(aoi_array)

Load AOI to dataframe

In [44]:
aoi_df_matrix = []
for path_array in aoi_cat_path_matrix:
    aoi_df_array = []
    for path in path_array:
        raw = pd.read_excel(path)
        tmp_df = pd.DataFrame(raw)
        aoi_df_array.append(tmp_df)
    aoi_df_matrix.append(aoi_df_array) 

Split DataFrames

In [45]:
df_BU_R = df.loc[(df["Comprehension"]=="BU") & (df["ProgrammingStyle"]=="R")]
df_BU_I = df.loc[(df["Comprehension"]=="BU") & (df["ProgrammingStyle"]=="I")]
df_TD_R = df.loc[(df["Comprehension"]=="TD") & (df["ProgrammingStyle"]=="R")]
df_TD_I = df.loc[(df["Comprehension"]=="TD") & (df["ProgrammingStyle"]=="I")]

df_array = [df_BU_R, df_BU_I, df_TD_R, df_TD_I]

Cast Data to Valid format

In [46]:
#cast data to valid format

def to_valid(string_points: str):
    string_points = string_points.strip()
    
    coordinate_str_array = string_points.split(" ")
    coordinates = []
    for coordinate_str in coordinate_str_array:
        coordinate = coordinate_str.split("-")
        coordinate = (int(coordinate[1]))
        coordinates.append(coordinate) 
    return coordinates

In [47]:
iterative = ['none', 'main', 'Iterative definition', 'Pre calculation', 'Iteration Condition', 'Iteration Step', 'Return Result']
recursive = ['none','main', 'Recursive definition', 'Pre calculation', 'Recursive Condition', 'Recursive Step', 'Return Result']
order = ['0', '1', '2', '3', '4', '5', '6']

In [48]:
def is_in(df, y):
    for _idx, row in df.iterrows():
        if row["startHeight"] <= y <= row["stopHeight"]:
            return row["Name"]
    return "none"

In [49]:
cols = ["startHeight", "stopHeight", "startWidth", "stopWidth", "Name"]

relation_tensor = []
for idx, df in enumerate(df_array):
    names = None
    if idx % 2 == 0:
        names = recursive
    else:
        names = iterative
        
    relation = []
    for i in range(len(names)):
        relation.append([0]*len(names))
    
    
    for row_idx, row in df.iterrows():
        algo_name = row["Algorithm"]
        aoi_df = aoi_df_matrix[algo_name_array.index(algo_name)][idx]
        click_data = row["ClickData"] 
        click_data = to_valid(click_data)
        labels = [is_in(aoi_df, coord) for coord in click_data]
        
        for i in range(len(labels)-1):
            from_node = names.index(labels[i])
            to_node = names.index(labels[i+1])
            
            relation[from_node][to_node] += 1
    relation_tensor.append(relation)

In [50]:
import numpy as np
for matrix in relation_tensor:
    for idx in range(len(matrix)):
        matrix[idx][idx] = 0
        
tensor = [np.array(matrix)[1:, 1:].tolist() for matrix in relation_tensor]
order = ['1', '2', '3', '4', '5', '6']

In [51]:
Chord(tensor[0], order, padding=0.1).to_html("BR_general.html")
Chord(tensor[1], order, padding=0.1).to_html("BI_general.html")

In [52]:
Chord(tensor[2], order, padding=0.1).to_html("TR_general.html")
Chord(tensor[3], order, padding=0.1).to_html("TI_general.html")

In [ ]:
from pathlib import Path

file = Path('BR_general.html')
file.write_text(file.read_text().replace('True', 'true'))

file = Path('BI_general.html')
file.write_text(file.read_text().replace('True', 'true'))

file = Path('TR_general.html')
file.write_text(file.read_text().replace('True', 'true'))

file = Path('TI_general.html')
file.write_text(file.read_text().replace('True', 'true'))


In [ ]:
names = ["Action", "Adventure", "Comedy", "Drama", "Fantasy", "Thriller"]

matrix = [
    [0, 5, 6, 4, 7, 4],
    [5, 0, 5, 4, 6, 5],
    [6, 5, 0, 4, 5, 5],
    [4, 4, 4, 0, 5, 5],
    [7, 6, 5, 5, 0, 4],
    [4, 5, 5, 5, 4, 0]
]

Chord(matrix, names).to_html()